## Table of Contents

* [Setup & Imports](#imports)
* [Reproducibility study](#4.1)
    * [Qualitative evaluation of counterfactual samples](#qual-eval-cf-samples) (HQC)
    * [Evaluating invariant classifiers](#eval-inv-clf) (ODR)
        * [Experiments on MNISTs](#repr-mnist)
        * [Experiments on ImageNet-mini](#repr-in-mini)
    * [Loss Ablation](#loss-ablation) (IBR)
* [Additional experiments and analyses](#add-expts)
    * [Explainability analysis on MNISTs](#expl-mnist)
    * [Explainability analysis on ImageNet-mini](#expl-in-mini)
    * [Robustness to OOD generalization](#ood)

## Setup & Imports  <a class="anchor" id="imports"></a>

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import repackage
repackage.up()

'/var/folders/d9/m81khvn17238ctpgh_j2522h0000gn/T'

In [3]:
import warnings
warnings.filterwarnings("ignore")

### Download data

The ImageNet-mini dataset needs to be downloaded from Kaggle. Please export your Kaggle credentials using the following command. The key is the Kaggle API key and can be found in your account settings.
```sh
export KAGGLE_USERNAME=<your_username>
export KAGGLE_KEY=<your_key>
```

Or alternatively, you can download your API key `kaggle.json` file and put it here `~/.kaggle/kaggle.json`.

> Note: Downloading all datasets takes about 20 mins and needs about 7GB of free space. In case the download fails at some point, you can re-run the cell, it will not download datasets already downloaded


In [ ]:
!python ../setup/download_datasets.py

This should download datasets for both `mnists` and `imagenet` tasks.

For MNISTs, the folder structure is as follows:
```sh
mnists/data
├── colored_mnist
└── textures
    ├── background
    └── object

4 directories
```

For ImageNet, the folder structure is as follows:
```sh
imagenet/data
├── cue_conflict
├── in-a
├── in-mini
├── in-sketch
├── in-stylized
└── in9

6 directories
```

### Download model weights

> Note: This takes < 5 mins to run and needs about 6GBs of free space. In case the download fails at some point, you can re-run the cell, it will not download model weights already downloaded.

In [ ]:
!python ../setup/download_weights.py

This will download the weights for all tasks.

```bash
imagenet/weights/
├── biggan256.pth
├── cgn.pth
├── u2net.pth
├── :
└── resnet50_from_scratch_model_best.pth.tar

4 files
```

## Section 4.1. Reproducibility study <a class="anchor" id="4.1"></a>

### Qualitative evaluation of counterfactual samples <a class="anchor" id="qual-eval-cf-samples"></a>

**Addressed Claim: High-quality counterfactuals (HQC)**


* Relevant section in the paper: Section 4.1.
* Relevant figures in the paper: Figure 2 and 3 

In [ ]:
#from counterfactual_mnist import main as mnist_main
from counterfactual_imagenet_og import main as imagenet_main_og

In [ ]:
# mnist_main(dataset_size=100, no_cfs=1)

In [ ]:
imagenet_main_og()

In [ ]:
from counterfactual_imagenet import main as imagenet_main_fake
imagenet_main_fake()

### Evaluating invariant classifiers <a class="anchor" id="eval-inv-clf"></a>

**Addressed Claim: Out-of-Distribution Robustness (ODR)**

#### Experiments on MNISTs <a class="anchor" id="repr-mnist"></a>

* Relevant tables in paper: Table 2

**Step 1**: Prepation - Run experiments on GPUs before visualizing results

> **Note**: It might take a long time to run this on a CPU machine (~40 mins). Instead, we would recommend
> running it on a GPU machine using the following instructions. That will run and generate all results
> and then you could run the cell below which will display all the results. We give instructions to run on a cluster managed by `Slurm`. In case you can get terminal access to GPU machines, you can run the individual scripts (look at the job files to check the commands).

**Running using cached models wherever necessary**

Follow the instruction to run it on a GPU on a cluster. 

1. We have created a job script `run_mnist.job`.
    Please change it appropriately, if needed, to run on a GPU.

2. Run the job script using 

    ```sh
    cd /path/to/repo/experiments/
    sbatch run_mnist.job
    ```
    You can check logs in `slurm_output_*.out` files.

3. After these steps are done, you can run the following cells that will display the result.

> This step takes about 20-25 mins to run on a GPU with 2 CPUs. Please let this step finish before running the cells.


**Re-running without using any cached models.**

The previous steps will generate CF data and load classification results from cache. If you want to run all these steps from scratch, you need to change `ignore_cache=True` in the following line in `mnist_pipeline.py` and follow the same steps as above.
```python
df = run_experiments(seed=0, ignore_cache=False)
```
You do not need to change the cell below.

**Step 2**: Replication of Table 2

In [ ]:
# from mnist_pipeline import run_experiments

In [ ]:
# # here, it is needed to pass `ignore_cache=False` since the results must be generated
# # from the previous step, thus you only need to run this as it is
# df = run_experiments(seed=0, show=False, ignore_cache=False)

In [ ]:
# df.index = ["Original", "GAN", "CGN", "Original + GAN", "Original + CGN"]

In [ ]:
# # show the results    
# df.astype(float).round(1)

#### Experiments on ImageNet-mini <a class="anchor" id="repr-in-mini"></a>

* Relevant tables in the paper: Table 3 and 4

**Step 1**: Prepation - Run experiments on GPUs before visualizing results

**Running using cached models wherever necessary**

Follow the instruction to run it on a GPU on a cluster. 

1. We have created a job script `run_imagenet.job`.
    Please change it appropriately, if needed, to run on a GPU.

2. Run the job script using 

    ```sh
    cd /path/to/repo/experiments/
    sbatch run_imagenet.job
    ```
    You can check logs in `slurm_output_*.out` files.

3. After these steps are done, you can run the following cells that will display the result.

> This step takes about 5 mins to run. Please let this step finish before running the cells.


**Re-running without using any cached models.**

The previous steps will generate CF data and load classification results from cache. If you want to run all these steps from scratch, you need to change `ignore_cache=True` and `generate_cf_data=True` in the following line in `experiments/imagenet_pipeline.py` and follow the same steps as above.

```python
metrics_clf, df_ood = run_experiments(seed=0, generate_cf_data=False, disp_epoch=34, ignore_cache=False)
```

> Note: This section involves generating counterfactual samples and training classifiers on IN-mini.
> Generating CF samples can take about 3.5 hours and training the classifier about 2 hours on a GPU.

**Step 2**: Results for Table 3 and 4 from the paper

In [2]:
from imagenet_pipeline import run_experiments
# temporarily showing results for 0th epoch
# metrics_clf, df_ood = run_experiments(seed=0, disp_epoch=34)
metrics_clf, df_ood = run_experiments(seed=0, disp_epoch=34, generate_cf_data=True, ignore_cache=True)


::::: Generating CF dataset :::::

Generating train dataset...
Saving data to imagenet/data/in-mini-train_trunc_0.5


  0%|          | 22/34745 [03:07<82:13:58,  8.53s/it] 


KeyboardInterrupt: 

In [ ]:
# construct Table 3 of the paper

heads = ["shape", "texture", "bg"]
table_3 = pd.DataFrame(
    None,
    columns=["Shape bias", "Top 1", "Top 5"],
    index=[f"IN-mini + CGN/{h}" for h in heads],
)
for i, h in enumerate(heads):
    table_3.at[f"IN-mini + CGN/{h}", "Shape bias"] = metrics_clf[f"shape_biases/{i}_m_{h}_bias"]
    table_3.at[f"IN-mini + CGN/{h}", "Top 1"] = metrics_clf[f"acc1/1_real"]
    table_3.at[f"IN-mini + CGN/{h}", "Top 5"] = metrics_clf[f"acc5/1_real"]

table_3["Shape bias"] *= 100.0
table_3 = table_3.astype(float).round(1)

In [ ]:
table_3

In [ ]:
# construct Table 4 of the paper
table_4 = pd.DataFrame(
    None,
    columns=["IN-9", "Mixed-same", "Mixed-rand", "BG-gap"],
    index=["IN-mini + CGN"],
)

col_to_key = {
    "IN-9": "in_9_acc1_original/shape_texture",
    "Mixed-same": "in_9_acc1_mixed_same/shape_texture",
    "Mixed-rand": "in_9_acc1_mixed_rand/shape_texture",
    "BG-gap": "in_9_gaps/bg_gap",
}

for c in table_4.columns:
    assert col_to_key[c] in metrics_clf
    key = col_to_key[c]
    table_4.at["IN-mini + CGN", c] = metrics_clf[key]

table_4 = table_4.astype(float).round(1)

In [ ]:
table_4

### Loss ablation <a class="anchor" id="loss-ablation"></a>

**Addressed Claim: Inductive Bias Requirements (IBR)**

* Relevant section in the paper: `Evaluating loss ablation` within Section 4.1.
* Relevant table in paper: Table 2

In [ ]:
# from ablation_study import run_experiments

# for loss_name, inception, avg_mask, sd_mask in run_experiments(ignore_cache=False):
#     print(f"for {loss_name} inception_score = {inception} and mu_mask = {avg_mask}")

### Additional experiments and analyses <a class="anchor" id="add-expts"></a>

#### Explainability analysis on MNISTs <a class="anchor" id="expl-mnist"></a>

* Relevant sections in the paper: 4.2.2
* Relevant figures in the paper: Figure 5 and 6

In [ ]:
# from mnist_analysis import run_analyses

In [ ]:
# run_analyses(
#     datasets=["colored_MNIST", "double_colored_MNIST", "wildlife_MNIST"],
#     debug=False,
#     show=True,
#     ignore_cache=False,
# )

#### Explainability analysis on ImageNet-mini <a class="anchor" id="expl-in-mini"></a>

* Relevant figures in the paper: Figure 7

In [ ]:
# from gradio_demo import init_gradio_module

In [ ]:
# cgn_gradio = init_gradio_module(launch=True, share=True)

#### Robustness to out-of-distribution generalization <a class="anchor" id="ood"></a>

* Relevant tables in the paper: Table 5

In [ ]:
# df_ood = df_ood.astype(float).round(2)

In [ ]:
# df_ood

### Appendix

#### Ablation on number of counterfactuals

In [ ]:
# def plot_mnist_ablation_on_cfs(
#         file,
#         datasets=["colored_MNIST", "double_colored_MNIST", "wildlife_MNIST"],
#     ):
#     CF_ratios = [1, 5, 10, 20]
#     dataset_sizes = [10000, 100000, 1000000]

#     with open(file, "r") as f:
#         results = json.load(f)

#     fig, axs = plt.subplots(1, 3, figsize=(16,5))
#     plt.setp(axs, xticks=[0, 1, 2], xticklabels=[r'$10^4$', r'$10^5$', r'$10^6$'])

#     for i, dataset in enumerate(datasets):
#         for CF_ratio in CF_ratios:
#             # Skip the CF_ratio of 20 for the colored MNIST dataset, as there are only 10 possible colors
#             # per shape.
#             if CF_ratio == 20 and dataset == "colored_MNIST":
#                 continue

#             line = []
#             for size in dataset_sizes:
#                 line.append(results[f"{dataset}_counterfactual_{size}_{CF_ratio}"])
#             axs[i].plot(np.arange(3), line, label=f'CF ratio = {CF_ratio}', marker='o')
#             axs[i].set_xlabel("Num Counterfactual Datapoints")
#             axs[i].set_ylabel("Test Accuracy (%)")
#             axs[i].grid(True)
#             axs[i].legend()
#         axs[i].set_title(datasets[i])

#     plt.savefig('../media/figures/figure7_reproduced.pdf', bbox_inches='tight')
#     plt.show()


In [ ]:
# results_path = "../experiments/results/cache/mnist_ablation_on_cfs.json"
# plot_mnist_ablation_on_cfs(results_path)